In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.12.0


In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
def preprocess(df, name):
    df_gyro = pd.DataFrame(columns=['time','GY(x)', 'GY(y)', 'GY(z)'])
    df_acc = pd.DataFrame(columns=['time','LA(x)', 'LA(y)', 'LA(z)'])
    
    df_gyro[['time','GY(x)', 'GY(y)', 'GY(z)']] = df[['TimeStamp','GY(x)', 'GY(y)', 'GY(z)']].copy()
#     df_gyro['name'] = df_gyro['name'].fillna(name)
    df_gyro = df_gyro.dropna()
    df_acc[['time','LA(x)', 'LA(y)', 'LA(z)']] = df[['TimeStamp','LA(x)', 'LA(y)', 'LA(z)']].copy()
#     df_acc['name'] = df_acc['name'].fillna(name)
    df_acc = df_acc.dropna()
    
    df_gyro['time'] = pd.to_datetime(df_gyro['time'], unit='ms')
    df_indexed_gyro = df_gyro.set_index('time')
    resampled_df_gyro = df_indexed_gyro.resample('5ms').mean().interpolate()
    resampled_df_gyro['name'] = name
    resampled_df_gyro.reset_index(inplace=True)
    
    df_acc['time'] = pd.to_datetime(df_acc['time'], unit='ms')
    df_indexed_acc = df_acc.set_index('time')
    resampled_df_acc = df_indexed_acc.resample('5ms').mean().interpolate()
#     resampled_df_acc['name'] = name
    resampled_df_acc.reset_index(inplace=True)
    
    # Inner join: Only the common values in both DataFrames are included in the merged DataFrame (inner_merged_df).
    
    inner_merged_df = pd.merge(resampled_df_acc, resampled_df_gyro, on='time', how='inner')
    
    return inner_merged_df

In [4]:
df_final = pd.DataFrame(columns=['name','time','LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
directory = os.path.join(os.getcwd(), 'straightWalk')
x=1
firstTime = True
for filename in os.listdir(directory):
    if (filename.endswith(".csv")):
        filepath = os.path.join(directory, filename)
        df_original = pd.read_csv(filepath)
        
#         num_rows_before = df_original.shape[0]
    
#         twenty = num_rows_before * 0.20
#         twenty = math.ceil(twenty)
        
#         #test = test.iloc[fivePercent:]
#         df_original_remove20 = df_original.iloc[:-twenty]
        
         
       
        df_processed = preprocess(df_original, filename[:-4])

        df_final = pd.concat([df_final, df_processed])
        
df_final

,name,time,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z)
0,alibadawy,1970-01-01 00:00:00.000,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138
1,alibadawy,1970-01-01 00:00:00.005,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157
2,alibadawy,1970-01-01 00:00:00.010,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000
3,alibadawy,1970-01-01 00:00:00.015,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288
4,alibadawy,1970-01-01 00:00:00.020,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030
...,...,...,...,...,...,...,...,...
5275,zoz,1970-01-01 00:00:26.375,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494
5276,zoz,1970-01-01 00:00:26.380,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391
5277,zoz,1970-01-01 00:00:26.385,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786
5278,zoz,1970-01-01 00:00:26.390,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156


In [5]:
df_Xtime = df_final.drop(['time'], axis = 1).copy()
df_Xtime.shape

(155302, 7)

In [6]:
label = LabelEncoder()
df_Xtime['label'] = label.fit_transform(df_Xtime['name'])
df_Xtime

,name,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,alibadawy,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138,0
1,alibadawy,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157,0
2,alibadawy,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000,0
3,alibadawy,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288,0
4,alibadawy,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030,0
...,...,...,...,...,...,...,...,...
5275,zoz,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494,30
5276,zoz,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391,30
5277,zoz,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786,30
5278,zoz,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156,30


In [7]:
label.classes_

array(['alibadawy', 'dinaelgendy', 'farah', 'ghazaly', 'habibatarabay',
       'hananabil', 'hashem', 'heidi', 'hussenhany', 'ismail', 'janah',
       'karen', 'mayar', 'mira', 'moataz', 'mohameddawoud',
       'mohamedelshoura', 'moustafa', 'nourhan', 'ola', 'rolla', 'salwa',
       'seifana', 'seifzakaria', 'tokhye', 'waed', 'yaseen', 'youssef',
       'youssefemad', 'yusuf', 'zoz'], dtype=object)

In [8]:
X = df_Xtime[['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)']].copy()
y = df_Xtime['label'].copy()

In [9]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
scaled_X['label'] = y.values

scaled_X

,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,-0.157996,0.116219,-0.109081,-0.070382,0.176505,0.142357,0
1,-0.120536,0.116679,-0.048151,-0.071206,0.181998,0.143630,0
2,-0.068498,0.113950,-0.067973,-0.072545,0.191841,0.139684,0
3,-0.045976,0.114420,-0.044167,-0.073060,0.201378,0.138793,0
4,-0.023454,0.114889,-0.020362,-0.067292,0.209007,0.134721,0
...,...,...,...,...,...,...,...
155297,-0.050031,0.129187,0.076822,0.083323,0.081973,0.380324,30
155298,-0.233456,0.004690,-0.064686,0.100625,0.025666,0.436450,30
155299,-0.383684,-0.097750,-0.177129,0.103714,-0.024689,0.499450,30
155300,-0.433164,-0.132987,-0.236030,0.093828,-0.059938,0.553668,30


In [10]:
import scipy.stats as stats

Fs = 200
frame_size = Fs*7
hop_size = Fs*1

In [11]:
def get_frames(df, frame_size, hop_size):
    N_FEATURES = 6

    frames = []
    labels = []
    
    last_value = df['label'].iloc[-1]
    for j in range (0,last_value+1): 
        filtered_df = df[df['label'] == j]
        filtered_df = filtered_df.drop('label', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            df_frame = filtered_df[i: i + frame_size].copy()
            
            

            label = j

            temp_frames = df_frame.to_numpy()
            
            frames.append(temp_frames)
            labels.append(label)

    frames = np.asarray(frames)
    labels = np.asarray(labels)
    
    return frames, labels
X, y = get_frames(scaled_X,frame_size, hop_size)
X.shape, y.shape
        

((574, 1400, 6), (574,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

In [13]:
X_train.shape, X_test.shape

((401, 1400, 6), (173, 1400, 6))

In [14]:
# Get the unique elements and their counts
unique_elements, counts = np.unique(y_train, return_counts=True)

# Print the unique elements and their counts
for element, count in zip(unique_elements, counts):
    print(f"Number {element} is repeated {count} times")

Number 0 is repeated 13 times
Number 1 is repeated 10 times
Number 2 is repeated 15 times
Number 3 is repeated 13 times
Number 4 is repeated 16 times
Number 5 is repeated 15 times
Number 6 is repeated 12 times
Number 7 is repeated 10 times
Number 8 is repeated 12 times
Number 9 is repeated 11 times
Number 10 is repeated 11 times
Number 11 is repeated 17 times
Number 12 is repeated 16 times
Number 13 is repeated 13 times
Number 14 is repeated 11 times
Number 15 is repeated 14 times
Number 16 is repeated 13 times
Number 17 is repeated 15 times
Number 18 is repeated 13 times
Number 19 is repeated 13 times
Number 20 is repeated 15 times
Number 21 is repeated 10 times
Number 22 is repeated 13 times
Number 23 is repeated 11 times
Number 24 is repeated 11 times
Number 25 is repeated 15 times
Number 26 is repeated 13 times
Number 27 is repeated 10 times
Number 28 is repeated 11 times
Number 29 is repeated 15 times
Number 30 is repeated 14 times


In [15]:
# chatgpt
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, activation='softmax'))

In [16]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv1d (Conv1D)                 (None, 1398, 32)              608         
                                                                           
 max_pooling1d (MaxPooling1D)    (None, 699, 32)               0           
                                                                           
 conv1d_1 (Conv1D)               (None, 697, 64)               6208        
                                                                           
 max_pooling1d_1 (MaxPooling1D)  (None, 348, 64)               0           
                                                                           
 conv1d_2 (Conv1D)               (None, 346, 128)              24704       
                                                                           
 max_pooling1d_2 (MaxPooling1D)  (None, 173, 128)              0    

In [17]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

In [18]:
history = model.fit(X_train, y_train, epochs = 20, verbose=1)

Epoch 1/20
13/13 [==============================] - 2s 84ms/step - loss: 3.8496 - accuracy: 0.0200
Epoch 2/20
13/13 [==============================] - 1s 85ms/step - loss: 3.3173 - accuracy: 0.1546
Epoch 3/20
13/13 [==============================] - 1s 83ms/step - loss: 2.9432 - accuracy: 0.2569
Epoch 4/20
13/13 [==============================] - 1s 83ms/step - loss: 2.1272 - accuracy: 0.4688
Epoch 5/20
13/13 [==============================] - 1s 84ms/step - loss: 1.2697 - accuracy: 0.6584
Epoch 6/20
13/13 [==============================] - 1s 83ms/step - loss: 0.6329 - accuracy: 0.8454
Epoch 7/20
13/13 [==============================] - 1s 84ms/step - loss: 0.3570 - accuracy: 0.9077
Epoch 8/20
13/13 [==============================] - 1s 83ms/step - loss: 0.1962 - accuracy: 0.9526
Epoch 9/20
13/13 [==============================] - 1s 83ms/step - loss: 0.1238 - accuracy: 0.9726
Epoch 10/20
13/13 [==============================] - 1s 87ms/step - loss: 0.0854 - accuracy: 0.9776
Epoch 11/

In [19]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

6/6 [==============================] - 0s 17ms/step - loss: 3.1133 - accuracy: 0.3468
Test loss:  3.1133. Test accuracy:  34.68%
